## Testing QuakeFlow on Alaska continuous data

In [23]:
from collections import defaultdict
import numpy as np
import pandas as pd
import time
import requests
import json
import obspy
from obspy.clients.fdsn import Client
import geopandas as gpd
from datetime import timedelta

In [3]:
import warnings
warnings.filterwarnings('ignore')

#### Create obspy catalog object from quakeml file
This in order to get a list of stations that were operating that month.

In [4]:
cat = obspy.core.event.read_events('XO_2018_10.quakeml')
print(cat)

437 Event(s) in Catalog:
2018-10-01T06:02:23.795000Z | +57.940, -153.929 | 2.1 ml | manual
2018-10-01T08:35:07.702000Z | +57.139, -156.563 | 2.5 ml | manual
...
2018-10-31T11:05:22.076000Z | +56.208, -148.992 | 2.4 ml | manual
2018-10-31T20:03:47.517000Z | +55.858, -149.523 | 2.4 ml | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'


#### Now let's get a list of the stations to pull data from (all the stations operating that month)

This will be in the form networkcode.stationcode.locationcode.channelbase

#### Note: let's throw out the pressure channels (code HDH)

In [8]:
# All the picks for the month:
picks=[p.picks for p in cat.events]
picks = sum(picks, [])

In [61]:
networks = [p.waveform_id.network_code for p in picks]
stations = [p.waveform_id.station_code for p in picks]
channels = [p.waveform_id.channel_code[0:2] + "*" for p in picks]
# Toss the pressure channels:
valueToBeRemoved = 'HD*'
channels = [value for value in channels if value != valueToBeRemoved]

networks= ['XO']

# Python 'f' strings are quite handy for string formatting:
sta_list = [f"{n}.{s}..{c[0:2]}" for n, s, c in zip(networks, stations, channels)]
sta_list = np.unique(sta_list)

#### Let's get the station information for those stations from IRIS


In [63]:
network = ",".join((np.unique(networks)).tolist())
channel = ",".join((np.unique(channels)).tolist())
station = ",".join((np.unique(stations)).tolist())
starttime=cat.events[0].origins[0].time

sta_metadata = Client("iris").get_stations(starttime = starttime, endtime  = starttime + timedelta(weeks=4),network = network,channel=channel,station=station,level='response',location='')
    
print(sta_metadata)

Inventory created at 2022-01-15T00:01:49.959000Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.48
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2018-10-01...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			XO
		Stations (89):
			XO.EP14 (Igiugig, AK)
			XO.EP15 (Kulic Lake, AK)
			XO.EP16 (Levelock, AK)
			XO.EP21 (Egegik, AK)
			XO.EP22 (Shoemaker Lodge, AK)
			XO.EP23 (Yantarni, AK)
			XO.ET17 (Naknek Lake, AK)
			XO.ET18 (Brooks Camp, AK)
			XO.ET19 (Valley10K Smokes, AK)
			XO.ET20 (Barrier Range, AK)
			XO.KD00 (Pasagshak Bay)
			XO.KD01 (Kalsin Bay)
			XO.KD02 (Anton Larsen Bay, AK)
			XO.KD04 (Santa Flavia Bay, AK)
			XO.KD05 (Uganik, AK)
			XO.KD12 (Anvil Lake, AK)
			XO.KS03 (Chiniak School, AK)
			XO.KS11 (McDonald Lagoon, AK)
			XO.KS13 (Akhiok School, AK)
			XO.KT06 (Harverster Island, AK)
			XO.KT07 (Larsen Bay, AK)
			XO.KT08 (Amook, AK)
			XO.KT09 (Uyak Bay, AK)
			XO.LA21 (LDEO OBS Deep APG/Hydro)
			XO.LA23 (LDEO 

#### Let's turn it into a more useful dataframe...

In [64]:
station_locs = defaultdict(dict)
for network in sta_metadata:
    for station in network:
        for chn in station:
            sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}"
            if sid in station_locs:
                station_locs[sid]["component"] += f",{chn.code[-1]}"
                station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
            else:
                component = f"{chn.code[-1]}"
                response = f"{chn.response.instrument_sensitivity.value:.2f}"
                dtype = chn.response.instrument_sensitivity.input_units.lower()
                tmp_dict = {}
                tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation(m)"] = (
                    chn.longitude,
                    chn.latitude,
                    chn.elevation,
                )
                tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                station_locs[sid] = tmp_dict

station_locs = pd.DataFrame.from_dict(station_locs, orient='index')
station_locs["id"] = station_locs.index

In [65]:
# Interactive visualization with geopandas geodataframe
gf = gpd.GeoDataFrame(station_locs.copy(), 
                      geometry=gpd.points_from_xy(station_locs.longitude, station_locs.latitude),
                      crs=4326,
                     )

gf.to_file('alaska_stations.json', driver='GeoJSON')

gf.explore()

### Now let's download some data!

In [66]:
client = Client("iris")
interval = 30 #s
# interval = 3600 #s

# for event in events:
def download(starttime, stations):
    '''
    For a given 'event' and 'stations' list download 30 second waveforms w/ 100Hz samping rate
    
    Output: obspy miniseed stream
    '''

    endtime = starttime + interval

    max_retry = 10
    stream = obspy.Stream()
    num_sta = 0
    for network in stations:
        for station in network:
            print(f"********{network.code}.{station.code}********")
            retry = 0
            while retry < max_retry:
                try:
                    tmp = client.get_waveforms(
                        network.code, station.code, "*", channel, starttime, endtime
                    )
                    for trace in tmp:
                        if trace.stats.sampling_rate != 100:
                            # print(trace)
                            trace = trace.interpolate(100, method="linear")
                    #      trace = trace.detrend("spline", order=2, dspline=5*trace.stats.sampling_rate)
                    #      stream.append(trace)
                    stream += tmp
                    num_sta += len(tmp)
                    break
                except Exception as err:
                    print("Error {}.{}: {}".format(network.code, station.code, err))
                    message = "No data available for request."
                    if str(err)[: len(message)] == message:
                        break
                    retry += 1
                    time.sleep(5)
                    continue
            if retry == max_retry:
                print(f"{fname}: MAX {max_retry} retries reached : {network.code}.{station.code}")
            
    # stream.attach_response(stations)
    # stream = stream.remove_sensitivity()
    return stream

In [67]:
def convert_mseed(mseed, station_locs):
    try:
        mseed = mseed.detrend("spline", order=2, dspline=5 * mseed[0].stats.sampling_rate)
    except:
        logging.error(f"Error: spline detrend failed at file {fname}")
        mseed = mseed.detrend("demean")
    mseed = mseed.merge(fill_value=0)
    starttime = min([st.stats.starttime for st in mseed])
    endtime = max([st.stats.endtime for st in mseed])
    mseed = mseed.trim(starttime, endtime, pad=True, fill_value=0)

    for i in range(len(mseed)):
        if mseed[i].stats.sampling_rate != sampling_rate:
            logging.warning(
                f"Resampling {mseed[i].id} from {mseed[i].stats.sampling_rate} to {sampling_rate} Hz"
            )
            mseed[i] = mseed[i].interpolate(sampling_rate, method="linear")

    order = ['3', '2', '1', 'E', 'N', 'Z']
    order = {key: i for i, key in enumerate(order)}
    comp2idx = {"3": 0, "2": 1, "1": 2, "E": 0, "N": 1, "Z": 2}

    nsta = len(station_locs)
    nt = max(len(mseed[i].data) for i in range(len(mseed)))
    data = []
    station_id = []
    t0 = []
    for i in range(nsta):
        trace_data = np.zeros([nt, n_channel], dtype=dtype)
        empty_station = True
        # sta = station_locs.iloc[i]["station"]
        sta = station_locs.index[i]
        comp = station_locs.iloc[i]["component"].split(",")
        if remove_resp:
            resp = station_locs.iloc[i]["response"].split(",")
            # resp = station_locs.iloc[i]["response"]
        print(sta)
        for j, c in enumerate(sorted(comp, key=lambda x: order[x[-1]])):

            resp_j = float(resp[j])
            
            
            if len(comp) != 3:  ## less than 3 component
                j = comp2idx[c]

            if len(mseed.select(id=sta + c)) == 0:
                print(f"Empty trace: {sta+c} {starttime}")
                continue
            else:
                empty_station = False

            tmp = mseed.select(id=sta + c)[0].data.astype(dtype)
            trace_data[: len(tmp), j] = tmp[:nt]

            if station_locs.iloc[i]["unit"] == "m/s**2":
                tmp = mseed.select(id=sta + c)[0]
                tmp = tmp.integrate()
                tmp = tmp.filter("highpass", freq=1.0)
                tmp = tmp.data.astype(dtype)
                trace_data[: len(tmp), j] = tmp[:nt]
            elif station_locs.iloc[i]["unit"] == "m/s":
                tmp = mseed.select(id=sta + c)[0].data.astype(dtype)
                trace_data[: len(tmp), j] = tmp[:nt]
            else:
                print(
                    f"Error in {station_locs.iloc[i]['station']}\n{station_locs.iloc[i]['unit']} should be m/s**2 or m/s!"
                )
            
            if remove_resp:
                trace_data[:, j] /= resp_j
                
        if not empty_station:
            data.append(trace_data)
            station_id.append(sta)
            t0.append(starttime.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3])

    data = np.stack(data)

    meta = {"data": data, "t0": t0, "station_id": station_id, "fname": station_id}
    
    
    return meta

Loop over the call to the download() function. It just takes the sta_metadata object and a starttime, in UTCdatetime.

In [47]:
meta = convert_mseed(mseed,station_locs)

XO.EP14..HH
XO.EP15..HH
XO.EP16..HH
XO.EP21..HH
XO.EP22..HH
XO.EP23..HH
XO.ET17..HH
XO.ET18..HH
Empty trace: XO.ET18..HH2 2018-10-01T06:02:23.785100Z
Empty trace: XO.ET18..HH1 2018-10-01T06:02:23.785100Z
XO.ET19..HH
XO.ET20..HH
XO.KD00..EH
XO.KD01..HH
XO.KD02..HH
XO.KD04..HH
XO.KD05..HH
XO.KD12..HH
XO.KS03..HH
XO.KS03..HN
XO.KS11..HH
XO.KS11..HN
XO.KS13..HH
XO.KS13..HN
XO.KT06..HH
XO.KT07..HH
XO.KT08..HH
XO.KT09..HH
XO.LA21..HD


KeyError: 'H'

In [69]:
# Let's just start with an hour

# INITIALIZE WITH PARAMETERS
base_time = starttime-30
sampling_rate = 100
n_channel = 3
dtype = "float32"
amplitude = True
remove_resp = True
PHASENET_API_URL = "http://phasenet.quakeflow.com"
config = {}
config["xlim_degree"] = [-170,-140]
config["ylim_degree"] = [50,65]
stations_json = station_locs.to_dict(orient="records")
config_gamma = {'xlim_degree': config["xlim_degree"], 
                'ylim_degree': config["ylim_degree"],
                'z(km)': [0, 41]}

phasenet_picks=[]
for i in range(1):
    # Download 30 s data from IRIS:
    mseed = download(starttime+(interval*i),sta_metadata)
    # Convert to numpy arrays:
    meta = convert_mseed(mseed,station_locs)
    batch = 4
    for j in range(0, len(meta["station_id"]), batch):
        req = {"id": meta['station_id'][j:j+batch],
        "timestamp": meta["t0"][j:j+batch],
        "vec": meta["data"][j:j+batch].tolist()}

    resp = requests.post(f'{PHASENET_API_URL}/predict', json=req)
    phasenet_picks.extend(resp.json())
    

********XO.EP14********
********XO.EP15********
********XO.EP16********
********XO.EP21********
********XO.EP22********
********XO.EP23********
********XO.ET17********
********XO.ET18********
********XO.ET19********
********XO.ET20********
********XO.KD00********
********XO.KD01********
********XO.KD02********
********XO.KD04********
********XO.KD05********
********XO.KD12********
********XO.KS03********
********XO.KS11********
********XO.KS13********
********XO.KT06********
********XO.KT07********
********XO.KT08********
********XO.KT09********
********XO.LA21********
********XO.LA23********
********XO.LA25********
********XO.LA26********
********XO.LA28********
********XO.LA29********
********XO.LA30********
********XO.LA32********
********XO.LA33********
********XO.LA34********
********XO.LA39********
********XO.LD36********
********XO.LD38********
********XO.LD40********
********XO.LD41********
********XO.LD44********
********XO.LD45********
********XO.LT01********
********XO.LT04*

#### Convert to numpy arrays

In [171]:
%%time

PHASENET_API_URL = "http://phasenet.quakeflow.com"

# req = {"id": meta["station_id"], 
#        "timestamp": meta["t0"],
#        "vec": meta["data"].tolist()}
# resp = requests.post(f'{PHASENET_API_URL}/predict', json=req)
# phasenet_picks = resp.json()

batch = 4
phasenet_picks = []
for j in range(0, len(meta["station_id"]), batch):
    req = {"id": meta["station_id"][j:j+batch], 
       "timestamp": meta["t0"][j:j+batch],
       "vec": meta["data"][j:j+batch].tolist(),
       "stations": stations_json,
       "config": config_gamma}

    resp = requests.post(f'{PHASENET_API_URL}/predict_phasenet2gamma2ui', json=req)
    phasenet_picks.extend(resp.json())


PhaseNet picks
CPU times: user 250 ms, sys: 21.6 ms, total: 272 ms
Wall time: 6.03 s


,id,timestamp,prob,amp,type
0,AV.ACH..BH,2018-10-01T09:42:19.528,0.976541,0.000076,p
1,AV.ACH..BH,2018-10-01T09:42:20.918,0.930866,0.000076,s
2,AV.ANCK..BH,2018-10-01T09:42:21.368,0.914994,0.000003,p
3,AV.ANCK..BH,2018-10-01T09:42:23.798,0.851268,0.000003,s
4,AV.KABU..BH,2018-10-01T09:42:19.308,0.981695,0.000031,p


In [245]:
# NOTE: optional you can run both phasenet and gamma with a single API call

PHASENET_API_URL = "http://phasenet.quakeflow.com"

config = {}
config["xlim_degree"] = [-170,-140]
config["ylim_degree"] = [50,65]
stations_json = station_locs.to_dict(orient="records")
config_gamma = {'xlim_degree': config["xlim_degree"], 
                'ylim_degree': config["ylim_degree"],
                'z(km)': [0, 41]}

req = {"id": meta["station_id"], 
       "timestamp": meta["t0"],
       "vec": meta["data"].squeeze().tolist(),
       "stations": stations_json,
       "config": config_gamma}

resp = requests.post(f'{PHASENET_API_URL}/predict_phasenet2gamma2ui', json=req)
result = resp.json()

Catalog:


,time,latitude,longitude,depth(m),magnitude,covariance
0,2018-10-01T09:42:15.237,58.17243,-155.232183,13715.249876,1.941594,"2.224,0.391,0.278"


Association:


,id,timestamp,prob,amp,type,event_idx,prob_gmma
0,AV.ACH..BH,2018-10-01T09:42:19.528,0.976541,0.000076,p,0,0.081009
1,AV.ACH..BH,2018-10-01T09:42:20.918,0.930866,0.000076,s,0,0.100742
2,AV.ANCK..BH,2018-10-01T09:42:21.368,0.914994,0.000003,p,0,0.059387
3,AV.ANCK..BH,2018-10-01T09:42:23.798,0.851268,0.000003,s,0,0.069766
4,AV.KABU..BH,2018-10-01T09:42:19.308,0.981695,0.000031,p,0,0.101909


### Now we save the QuakeFlow results to a json file:

In [248]:
save_file = 'result_example'

with open(save_file+'.json', 'w') as f:
    json.dump(result, f)